## Version Description
In this version, we deploy a Hybrid PINN to predict only one correction factor for the MTR ( MTR-600 ) by the neural network. We implement line search ([Secant Method](https://nickcdryan.com/2017/09/13/root-finding-algorithms-in-python-line-search-bisection-secant-newton-raphson-boydens-inverse-quadratic-interpolation-brents/)) to adaptively find the learning rate.
* Mini Batch training with `batch size = 2` is used.
* Two extra features `x1` and `x2` are engineered and the input features used are `['x1','x2']` only. `TZ6_flow` is used as a seperate input not part of `X` (set of all features in train dataset).
* **Linear** (Identity) Activation function is used in the **hidden and output layers**.

In [1]:
from IPython.core.display import display, HTML,display_html
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
# import required libraries
import random
import pandas as pd
import numpy as np

from functools import reduce
import sys

#Set some numpy print options for displaying numpy arrays to fit maximum width of cell
np.set_printoptions(precision=3, edgeitems=30, linewidth=1000,formatter=dict(float=lambda x: "%.3g" % x))

# Disable Warnings for chained assignments Eg:Setting with Copy Warning
pd.options.mode.chained_assignment = None

from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show, output_file,output_notebook
output_notebook() # Set to output the plot in the notebook

Loading BokehJS ...

In [3]:
np.random.seed(42)

# import LTR and corresponding Restrictor dataframe
Restrictor_data = pd.read_csv('D:/OneDrive/Documents/Airbus/GitHub/ML_A350_Restrictors/TZ6/data/Zone6_RestrictorData_LTR.csv')
LTR_data = pd.read_csv('D:/OneDrive/Documents/Airbus/GitHub/ML_A350_Restrictors/TZ6/data/Zone6_LTRData.csv')
obs_HOVs = pd.read_csv('../data/LTR_dataset.csv', usecols = ['HoV'])[['HoV']]
LTR_data[['AMBP','MIXP']] *= 100 # Convert hPa to Pa
zero_MIXT_hoV_idx = LTR_data.index[LTR_data['MIXT'] == 20].tolist() # Points where originally, MIXT == 0 and substituted with 20° C
LTR_data[['MIXT','AMBT']] += 273.15 # Convert Celcius to Kelvin
LTR_data[['TZ6_Flow']] /= 1000 # Convert l/s to cu.m/s
print('Total LTR points:',len(LTR_data))

## Calculate Density = (0.5* Pmix + Pamb) / (R * Tmix) where R = 287 is the Gas constant in Ideal Gas Law
idx = LTR_data.columns.get_loc("AMBT")
LTR_data.insert(loc=(idx+1), column='Density(kg/m³)', value=np.round((0.5 * LTR_data['MIXP'] + LTR_data['AMBP'] ) / (LTR_data['MIXT']*287),4))
target_df = reduce(lambda left,right: pd.merge(left,right,left_on='HoV',right_on='HoV'), [LTR_data,Restrictor_data])
target_df['HoV'] = obs_HOVs.values
target_df.head()

Total LTR points: 34


,HoV,CAOLH_C66-C68,CAOLH_C68-C70,CAOLH_C70-C72,CAOLH_C72-C74,CAOLH_C74-C76,CAOLH_C76-C78,CAORH_C66-C68,CAORH_C68-C70,CAORH_C70-C72,...,R632_HS1,R633_HS1,R634_HS1,R635_HS1,R636_HS1,R637_HS1,R638_HS1,R639_HS1,R640_HS1,R641_HS1
0,A1,36.071000,38.802000,36.796000,36.794000,39.100000,38.318000,37.055000,39.596000,38.333000,...,41,41,41,41,41,41,41,41,41,41
1,A2,36.061000,38.697000,37.156000,36.738000,38.957000,38.326000,36.501000,40.797000,39.284000,...,41,41,41,41,41,41,41,41,41,41
2,A3,37.504234,40.911735,38.412733,37.859887,36.585486,38.504564,36.161971,37.068225,37.893405,...,41,41,41,41,41,41,41,41,41,41
3,A4,36.383413,39.101836,38.110486,37.597728,36.190671,37.706898,36.668551,37.759174,38.497755,...,41,41,41,41,41,41,41,41,41,41
4,A5,36.897000,39.925000,38.704000,38.175000,40.591000,39.913000,37.641000,38.309000,38.600000,...,41,41,41,41,41,41,41,41,41,41


In [4]:
V_max_org = target_df['TZ6_Flow'].max()
print('Maximum TZ6 Flow:',V_max_org)

Maximum TZ6 Flow: 0.7640123719999998


In [5]:
MIXP = [1800, 2600, 3400]
# Define Restrictor Labels
MTR = 'R600_HD'
CLRS = ['R610_HS1','R611_HS1','R612_HS1','R613_HS1']
CAORS = ['R620_HS1','R620_HS2','R620_HS3','R621_HS1','R621_HS2','R621_HS3']
MHRS = CLRS + CAORS
LAORS = ['R630_HS1','R631_HS1','R632_HS1','R633_HS1','R634_HS1','R635_HS1','R636_HS1','R637_HS1','R638_HS1','R639_HS1','R640_HS1','R641_HS1']

In [6]:
# Duct Areas
MTR_DuctArea = 0.03464 # in sq.m
MHR_Duct_Areas = {'R610_HS1':0.01767,'R611_HS1':0.01767,'R612_HS1':0.02461,'R613_HS1':0.02461,
                 'R620_HS1':0.01090,'R620_HS2':0.00470,'R620_HS3':0.01085,'R621_HS1':0.01085,'R621_HS2':0.00470,'R621_HS3':0.01090}

## Define Initial Correction factor values from CFD
# Correction Factor for Duct Part `TZ6_zmix:
TZ6_zmix_cf = 1 # Start with initial c_f =1 

#3D Correction Factor for MTR
MTR_cf = 1.1664876228437


#3D Correction Factors for MultiHole Restrictors
MHR_cf = {'R610_HS1':0.740773983383846,'R611_HS1':0.740205245723713,'R612_HS1':0.83879652680329,'R613_HS1':0.81775586281569,
          'R620_HS1':0.757121843184036,'R620_HS2':0.773919779037984,'R620_HS3':0.789028593182055,
          'R621_HS1':0.714811824527744,'R621_HS2':0.866271631648556,'R621_HS3':0.822447592395462}

### Calculate Loss-Coefficient $\zeta$
**For Single Hole Restrictor:**<br>

    f0_f1 = A_Circular(Hole_Diameter) / Area_Overall
    l_cross = Thickness / Hole_Diameter
    Zeta_dash = 0.13 + 0.34 * 10 ^ -(3.4 * l_cross + 88.4 * l_cross ^ 2.3)
    Zeta_Single_Hole_Thick_Chamfered = ((1 - f0_f1 + (Zeta_dash ^ 0.5) * (1 - f0_f1) ^ 0.375) ^ 2) * f0_f1 ^ -2

In [7]:
def SHR_Zeta_3D(n_holes,hole_dia,MTR_DuctArea,cf):
    '''
    Computes the Zeta with 3D Correction Factor (cf) for Single Hole Retrictors
    '''
    MTR_New_Area = n_holes * (np.pi/4) * (hole_dia / 1000)**2 # Divide dia by 1000 to convert mm to m
    f0_f1 = MTR_New_Area/MTR_DuctArea
    l_cross = 1/hole_dia
    zeta_dash = 0.13 + 0.34 * 10**(-(3.4 * l_cross + 88.4 * l_cross**2.3))
    zeta_SHR_1D = ((1 - f0_f1 + (zeta_dash**0.5) * (1 - f0_f1)**0.375)**2) * f0_f1**(-2) # 1D Zeta
    zeta_SHR_3D = zeta_SHR_1D * cf # Zeta with 3D Correction Factor    
    return MTR_New_Area,zeta_SHR_3D

In [8]:
target_df[MTR+'_Area'],target_df[MTR+'_Zeta3D'] = zip(*[SHR_Zeta_3D(1,dia,MTR_DuctArea,MTR_cf) for dia in target_df[MTR]])
target_df[[MTR,'R600_HD_Area','R600_HD_Zeta3D']].head()

,R600_HD,R600_HD_Area,R600_HD_Zeta3D
0,148,0.017203,4.949375
1,149,0.017437,4.730821
2,152,0.018146,4.127180
3,154,0.018627,3.764504
4,148,0.017203,4.949375


**For Multi Hole Restrictors:**<br>

    Area_Free = Number_of_Holes * A_Circular(Hole_Diameter)
    f0_f1 = Area_Free / Area_Overall
    l_cross = Thickness / Hole_Diameter
    phi = 0.25 + (0.535 * l_cross ^ 8) / (0.05 + l_cross ^ 7)
    tau = (2.4 - l_cross) * 10 ^ (-phi)
    Zeta_Multi_Hole = (0.5 * (1 - f0_f1) ^ 0.75 + tau * (1 - f0_f1) ^ 1.375 + (1 - f0_f1) ^ 2 + 0.02 * l_cross) / f0_f1 ^ 2

In [9]:
def MHR_Zeta_3D(nr_holes,hole_dia,MHR_DuctArea,cf):
    '''
    Computes the Zeta with 3D Correction Factor (cf) for Multi Hole Restrictors
    '''
    MHR_New_Area = nr_holes * (np.pi/4) * (hole_dia / 1000)**2 # Divide dia by 1000 to convert mm to m
    f0_f1 = MHR_New_Area/MHR_DuctArea
    l_cross = (0.00144*1000)/hole_dia
    phi = 0.25 + (0.535 * l_cross**8) / (0.05 + l_cross**7)
    tau = (2.4 - l_cross) * 10**(-phi)
    zeta_MHR_1D = (0.5 * (1 - f0_f1)**0.75 + tau * (1 - f0_f1)**1.375 + (1 - f0_f1)**2 + 0.02 * l_cross) / f0_f1**2 # 1D Zeta     
    zeta_MHR_3D = zeta_MHR_1D * cf # Zeta with 3D Correction Factor    
    return MHR_New_Area,zeta_MHR_3D

In [10]:
# Calculate Zeta for Multi-Hole Restrictors
for clr in CLRS:
    if (clr == 'R610_HS1') | (clr == 'R611_HS1'):
        hole_dia = 8
    else:
        hole_dia = 10
    MHR_nr_holes = target_df[clr].values
    target_df[clr+'_Area'], target_df[clr+'_Zeta3D'] = zip(*[MHR_Zeta_3D(ele,hole_dia,MHR_Duct_Areas[clr],MHR_cf[clr]) for ele in MHR_nr_holes])
    
for caor in CAORS:
    CAOR_nr_holes = target_df[caor].values
    target_df[caor+'_Area'],target_df[caor+'_Zeta3D'] = zip(*[MHR_Zeta_3D(ele,8,MHR_Duct_Areas[caor],MHR_cf[caor]) for ele in CAOR_nr_holes])

**Store Loss Coefficient ($\zeta$) Values and Names for all Restrictors:**

In [11]:
# Gather col_names and their corresponding column indices
zeta_col_names = [col for col in target_df.columns if 'Zeta3D' in col]
zeta_col_names_idx = [target_df.columns.get_loc(col) for col in zeta_col_names if col in target_df]
area_col_names = [col for col in target_df.columns if 'Area' in col]
area_col_names_idx = [target_df.columns.get_loc(col) for col in area_col_names if col in target_df]
pt_var_idx = [target_df.columns.get_loc(col) for col in ['MIXP','AMBP','AMBT'] if col in target_df]
req_flow_rates_idx = [target_df.columns.get_loc(col) for col in ['TZ6_Flow'] if col in target_df]
# print(zeta_col_names_idx)
print(zeta_col_names)

['R600_HD_Zeta3D', 'R610_HS1_Zeta3D', 'R611_HS1_Zeta3D', 'R612_HS1_Zeta3D', 'R613_HS1_Zeta3D', 'R620_HS1_Zeta3D', 'R620_HS2_Zeta3D', 'R620_HS3_Zeta3D', 'R621_HS1_Zeta3D', 'R621_HS2_Zeta3D', 'R621_HS3_Zeta3D']


In [12]:
LTR_zeta_df_col_idx = sorted([0] + req_flow_rates_idx + pt_var_idx + zeta_col_names_idx)
target_df.iloc[:,LTR_zeta_df_col_idx].to_csv('data_output/LTR_Zeta_df.csv', index=False)

In [13]:
LTR_zeta_df = target_df[['HoV','MIXP','AMBP','AMBT','TZ6_Flow']+zeta_col_names]
final_df = target_df[['HoV','MIXP','Density(kg/m³)','R600_HD','R600_HD_Area','R600_HD_Zeta3D','TZ6_Flow']]
final_df.reset_index(drop=True,inplace = True)
final_df.head()

,HoV,MIXP,Density(kg/m³),R600_HD,R600_HD_Area,R600_HD_Zeta3D,TZ6_Flow
0,A1,2600.0000,1.2207,148,0.017203,4.949375,0.738083
1,A2,2600.0000,1.2228,149,0.017437,4.730821,0.737775
2,A3,2606.1928,1.2090,152,0.018146,4.127180,0.740980
3,A4,2599.8998,1.2368,154,0.018627,3.764504,0.732112
4,A5,2600.0000,1.2380,148,0.017203,4.949375,0.741828


### Feature Engineering

In [14]:
final_df['xf1'] = ( final_df['MIXP']/final_df['R600_HD_Zeta3D'] ) * (2/final_df['Density(kg/m³)']) * (MTR_DuctArea/ final_df['TZ6_Flow'])**2
final_df['xf2'] = ( 1 /final_df['R600_HD_Zeta3D'] )
final_df.drop(columns = ['MIXP','Density(kg/m³)','R600_HD','R600_HD_Zeta3D','R600_HD_Area','TZ6_Flow'], inplace = True)

### Train Data Preperation

In [15]:
input_features = ['xf1','xf2']
## Using un-scaled values
train_data = final_df[input_features]
print("Number of training samples:", len(train_data))
train_data

Number of training samples: 34


,xf1,xf2
0,1.895786,0.202046
1,1.981615,0.211380
2,2.282967,0.242296
3,2.500234,0.265639
4,1.850468,0.202046
5,1.869475,0.202046
6,1.899464,0.202046
7,2.398352,0.253672
8,2.319517,0.242296
9,2.044012,0.221181


In [16]:
# Store indices of train, test and validation data
train_data_idx = train_data.index.values.tolist()
train_R600HD_series = target_df['R600_HD'].iloc[train_data_idx]
X_train = train_data.values

### Predict $c_f$ with Physics Informed Neural Network

In [17]:
# Define custom MSE function
MSE = lambda y_hat,y_true: np.mean((np.asarray(y_hat)-np.asarray(y_true))**2)

**Train the Neural Network**

In [18]:
### Set the hyperparameters here ###
iterations = 55
lr0, lr1 = 0, 1
hidden_nodes = 1 ### Important Parameter
output_nodes = 1
batch_sample_size = 2
MTR_epsi = np.float64(MTR_cf * 0.01)

In [19]:
from Hybrid_PINN_v2_1 import PINN

In [20]:
%%time
N_i = X_train.shape[1] # no. of features
network = PINN(N_i, hidden_nodes, output_nodes, MTR_epsi)

MSE_flowloss_hist = []
err_loss_hist = []
c_f_hist = {'HoV':[],'c_f_hat':[],'FlowRate_Diff_(LTR-FDDN)':[],'ConvergedEpoch':[],'f1':[],'f2':[]}

# Set Initial Weights and Gradient Updates
w_ih, w_ho, dw_i_h, dw_h_o = None, None, None, None

for ii in range(iterations):    
    # Go through one record at a time from the training data set
    print('\nEPOCH {} - Length of TRAIN Dataset:'.format(ii), len(train_data_idx), ',.... REMAINING TRAINING POINTS:', train_data_idx )    
    batch_idx = np.random.choice(train_data_idx, replace = False, size = batch_sample_size) # train_data_idx
    X_train = train_data[input_features].loc[batch_idx].values
    dia_train = train_R600HD_series.loc[batch_idx]
    print('Current Indices in Batch:',batch_idx,'\nOld_Zeta:',list(target_df[[MTR+'_Zeta3D']].loc[batch_idx].values))    
    
    ## Initialize with default weights    
    w0_i_h, w0_h_o    = network.calc_new_weights(lr0, delta_weights_i_h = dw_i_h, delta_weights_h_o = dw_h_o, w_ih = w_ih, w_ho = w_ho)
    print('\nTRAINING ITERATION {} - w_0:'.format(ii), list(w0_i_h) + list(w0_h_o))                                                                        
    V_LTR_true, V_hat_FDDN, V_hat_epsi_FDDN, cf_array, hidden_outputs, _ = network.train(X_train, dia_train, batch_idx, w0_i_h, w0_h_o, MTR_DuctArea, target_df, zeta_col_names)
    _, dE_dw_ih0, dE_dw_ho0 = network.gradient_update(V_LTR_true, V_hat_FDDN , V_hat_epsi_FDDN, cf_array, X_train, hidden_outputs, dw_i_h, dw_h_o, w0_h_o, V_max_org)    
    f0 = network.cust_f_alpha(dE_dw_ih0, dE_dw_ho0, dE_dw_ih0, dE_dw_ho0 )        
    print('f0:', f0)
    w1_i_h, w1_h_o = network.calc_new_weights(lr1, dE_dw_ih0, dE_dw_ho0, w_ih = w0_i_h, w_ho = w0_h_o)
    V_LTR_true, V_hat_FDDN, V_hat_epsi_FDDN, cf_array, hidden_outputs, _ = network.train(X_train, dia_train, batch_idx, w1_i_h, w1_h_o, MTR_DuctArea, target_df, zeta_col_names)
    _,dE_dw_ih1, dE_dw_ho1 = network.gradient_update(V_LTR_true, V_hat_FDDN, V_hat_epsi_FDDN, cf_array, X_train, hidden_outputs, dw_i_h, dw_h_o, w1_h_o, V_max_org )
    f1 = network.cust_f_alpha(dE_dw_ih0, dE_dw_ho0, dE_dw_ih1, dE_dw_ho1)        
    print('f1:', f1)
    lr2 = network.calc_lr2( f0, f1, lr0, lr1)    
    w2_i_h, w2_h_o = network.calc_new_weights(lr2, dE_dw_ih0, dE_dw_ho0, w_ih = w0_i_h, w_ho = w0_h_o)    
    V_LTR_true, V_hat_FDDN, V_hat_epsi_FDDN, cf_array, hidden_outputs, break_flag_arr = network.train(X_train, dia_train, batch_idx, w2_i_h, w2_h_o, MTR_DuctArea, target_df, zeta_col_names)    
    error, dE_dw_ih2, dE_dw_ho2 = network.gradient_update(V_LTR_true, V_hat_FDDN, V_hat_epsi_FDDN, cf_array, X_train, hidden_outputs, dw_i_h, dw_h_o, w2_h_o, V_max_org)
    f2 = network.cust_f_alpha(dE_dw_ih0, dE_dw_ho0, dE_dw_ih2, dE_dw_ho2)        
    print('f2:', f2)
    w_ih, w_ho = w2_i_h, w2_h_o
    dw_i_h, dw_h_o = dE_dw_ih2, dE_dw_ho2
    
    # Print out the training progress        
    MSE_flowloss = MSE(V_hat_FDDN, V_LTR_true) # MSE between FDDN predicted flow rate and true flow rate on UN-scaled original values in cu.m/s
    sys.stdout.write("\rProgress: {:2.1f}".format(100 * ii/float(iterations))+ "% ... MSE FLowloss: " + str(round(MSE_flowloss,4))[:5] + " ... Error Function loss: " + str(round(error,4))) # For truncated version of the error
    print('... Training Iteration:',ii)    
    sys.stdout.flush()
    MSE_flowloss_hist.append(MSE_flowloss)
    err_loss_hist.append(error) # For truncated version of the error        
    if (1 in break_flag_arr): # To remove converged HoV
        if(0 in list(batch_idx)):
            row_ids = [0] # Manually include [0] index if it is in the current batch
        else:
            row_ids = list(filter(lambda e: e != 0, list(batch_idx * np.asarray(break_flag_arr)))) #Multiply row_ID list with bf_list and remove 0s
        HoVs = target_df[['HoV']].loc[row_ids].values.tolist()
        final_cfs = list(filter(lambda e: e != 0, list(cf_array * np.asarray(break_flag_arr)))) #Multiply MTR_cf_array with bf_list and remove 0s        
        V_LTR_final = list(filter(lambda e: e != 0, list(V_LTR_true.flatten('F') * np.asarray(break_flag_arr))))
        V_FDDN_final = list(filter(lambda e: e != 0, list(V_hat_FDDN.flatten('F') * np.asarray(break_flag_arr))))        
        print('REMOVING CONVERGED HoVs:',HoVs,'Rows Dropped:',row_ids)
        print("NN_CF_Array:", cf_array)
        print("Final CFs to Append:", final_cfs)
        for hov, cf, V_LTR, V_FDDN in zip(HoVs, final_cfs, V_LTR_final, V_FDDN_final):
            c_f_hist['HoV'].append(hov[0])
            c_f_hist['c_f_hat'].append(cf)
            c_f_hist['FlowRate_Diff_(LTR-FDDN)'].append(V_LTR - V_FDDN )
            c_f_hist['ConvergedEpoch'].append(ii)
            c_f_hist['f1'].append(f1)
            c_f_hist['f2'].append(f2)
        train_data_idx = list(set(train_data_idx).difference(set(row_ids)))
        if (len(train_data_idx) < batch_sample_size): # reduce batch size when available data is less than batch size            
            batch_sample_size = batch_sample_size - len(row_ids)             
    if (len(train_data_idx) == 0) or (ii == iterations - 1):
        sys.stdout.write('TRAINING PROCESS COMPLETED -')
        if (len(train_data_idx) == 0):
            print(' All HoVs Converged')
        elif (ii == iterations - 1):
            print(' Set Nr. of Training Epochs Reached')
        break

Initial Weights_i_h: [array([1]), array([0.1])]
Initial Weights_h_o: [array([1])]

EPOCH 0 - Length of TRAIN Dataset: 34 ,.... REMAINING TRAINING POINTS: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
Current Indices in Batch: [15 19] 
Old_Zeta: [array([4.95]), array([4.95])]

CALC NEW WEIGHTS - Learning Rate Used: 0
CALC NEW WEIGHTS - delta_weights_input_i_h_o: [array([0]), array([0]), array([0])]
CALC NEW WEIGHTS - new_weights_i_h: [array([1]), array([0.1])]
CALC NEW WEIGHTS - new_weights_h_o: [array([1])]

TRAINING ITERATION 0 - w_0: [array([1]), array([0.1]), array([1])]

NN output (C_f): 2.3465702780332913
New_Zeta: 9.956433919732403
FDDN Solver Output: [[60 28.6 90.5 59.2 31.3 96.4 9.48 20.5 20.4 20.5 20 19.8 9.26 20.1 20.3 20 19.7 19.7]]
New_Zeta_epsi: 10.005927672567127
FDDN Solver Output: [[59.9 28.6 90.3 59.1 31.2 96.3 9.46 20.4 20.4 20.4 20 19.7 9.24 20.1 20.3 19.9 19.7 19.7]]
Row ID: 15
HoV: ['E

FDDN Solver Output: [[76.4 36.5 115 75.4 39.8 123 12.2 26.3 26.3 26.3 25.8 25.4 11.8 25.6 25.8 25.4 25.1 25.1]]
New_Zeta_epsi: 4.004988709488766
FDDN Solver Output: [[76.3 36.4 115 75.2 39.7 123 12.2 26.3 26.2 26.3 25.7 25.4 11.7 25.6 25.8 25.4 25.1 25.1]]
Row ID: 10
HoV: ['C5']
Flow Rate Difference (LTR - FDDN): -0.020266183323459086 cu.m/s

NN output (C_f): 1.1389502960980566
New_Zeta: 3.849031795965704
FDDN Solver Output: [[78.6 37.5 119 76.2 40.3 124 12.3 26.5 26.5 26.5 26 25.6 11.9 25.9 26.1 25.7 25.4 25.4]]
New_Zeta_epsi: 3.888452725617399
FDDN Solver Output: [[78.5 37.4 118 76 40.2 124 12.3 26.5 26.4 26.5 25.9 25.6 11.9 25.9 26.1 25.6 25.3 25.3]]
Row ID: 7
HoV: ['C2']
Flow Rate Difference (LTR - FDDN): -0.01663637647545224 cu.m/s
GRADIENT_UPDATE - dw_ih: [array([-0.0124]), array([-0.0127])] <class 'numpy.ndarray'> (2, 1)
GRADIENT_UPDATE - dw_ho [array([0])] <class 'numpy.ndarray'> (1, 1)
f2: -0.00048558972207038816
Progress: 1.8% ... MSE FLowloss: 0.000 ... Error Function loss: 

FDDN Solver Output: [[76 36.3 115 75 39.6 122 12 25.9 25.8 25.9 25.3 25 11.7 25.5 25.7 25.3 25 25]]
Row ID: 6
HoV: ['C1']
Flow Rate Difference (LTR - FDDN): -0.003484317622293731 cu.m/s

NN output (C_f): 0.9726700428128867
New_Zeta: 4.127012558552657
FDDN Solver Output: [[76.1 36.3 115 75.1 39.7 122 12 25.9 25.9 25.9 25.4 25 11.7 25.5 25.7 25.3 25 25]]
New_Zeta_epsi: 4.1765063113873815
FDDN Solver Output: [[75.9 36.2 114 74.9 39.6 122 12 25.9 25.8 25.9 25.3 25 11.7 25.5 25.7 25.3 24.9 25]]
Row ID: 14
HoV: ['E1']
Flow Rate Difference (LTR - FDDN): -0.007403862496130897 cu.m/s
GRADIENT_UPDATE - dw_ih: [array([-0.00227]), array([-0.00228])] <class 'numpy.ndarray'> (2, 1)
GRADIENT_UPDATE - dw_ho [array([0])] <class 'numpy.ndarray'> (1, 1)
f1: 1.3144327315170007e-05
Learning Rate 2: 4.711366651185303

CALC NEW WEIGHTS - Learning Rate Used: 4.711366651185303
CALC NEW WEIGHTS - delta_weights_input_i_h_o: [array([-0.00288]), array([-0.0029]), array([0])]
CALC NEW WEIGHTS - Inputs Passed Weight

FDDN Solver Output: [[76.5 36.5 115 74.2 39.2 121 11.9 25.6 25.6 25.6 25.1 24.7 11.6 25.2 25.4 25 24.7 24.7]]
New_Zeta_epsi: 4.1811670721831815
FDDN Solver Output: [[76.3 36.4 115 74 39.1 121 11.8 25.6 25.5 25.6 25 24.7 11.6 25.2 25.4 25 24.7 24.7]]
Row ID: 30
HoV: ['S4']
Flow Rate Difference (LTR - FDDN): -0.003464908226178065 cu.m/s

NN output (C_f): 0.9521919307223472
New_Zeta: 4.04012448546191
FDDN Solver Output: [[76.1 36.3 115 75 39.6 122 12 25.9 25.9 25.9 25.4 25 11.7 25.5 25.7 25.3 25 25]]
New_Zeta_epsi: 4.0896182382966355
FDDN Solver Output: [[75.9 36.2 114 74.8 39.5 122 12 25.8 25.8 25.9 25.3 25 11.7 25.4 25.7 25.2 24.9 24.9]]
Row ID: 5
HoV: ['A6']
Flow Rate Difference (LTR - FDDN): 0.0013156892887473681 cu.m/s
% Diff between flowrates 0.1771077252028339 %
GRADIENT_UPDATE - dw_ih: [array([-0.00177]), array([-0.00158])] <class 'numpy.ndarray'> (2, 1)
GRADIENT_UPDATE - dw_ho [array([0])] <class 'numpy.ndarray'> (1, 1)
f0: 5.632354067704753e-06

CALC NEW WEIGHTS - Learning Rate 

FDDN Solver Output: [[79 37.7 119 76.5 40.4 125 12.5 26.9 26.8 26.9 26.3 26 12.1 26.3 26.5 26 25.7 25.7]]
New_Zeta_epsi: 3.9617882079101236
FDDN Solver Output: [[78.8 37.6 119 76.4 40.4 124 12.4 26.8 26.8 26.8 26.3 25.9 12 26.2 26.4 26 25.7 25.7]]
Row ID: 23
HoV: ['M2']
Flow Rate Difference (LTR - FDDN): -0.0010070226817667782 cu.m/s
% Diff between flowrates 0.13172030872218857 %
GRADIENT_UPDATE - dw_ih: [array([5.33e-05]), array([4.78e-05])] <class 'numpy.ndarray'> (2, 1)
GRADIENT_UPDATE - dw_ho [array([0])] <class 'numpy.ndarray'> (1, 1)
f2: 2.912037322709297e-07
Progress: 10.9% ... MSE FLowloss: 0.0 ... Error Function loss: 0.0... Training Iteration: 6
REMOVING CONVERGED HoVs: [['S1'], ['M2']] Rows Dropped: [27, 23]
NN_CF_Array: [1.31 1.22]
Final CFs to Append: [1.3124771122347556, 1.2159543446523418]

EPOCH 7 - Length of TRAIN Dataset: 28 ,.... REMAINING TRAINING POINTS: [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 28, 31, 32, 33]
Current

...

FDDN Solver Output: [[73.8 35.2 111 72.8 38.5 119 11.7 25.1 25.1 25.2 24.6 24.3 11.4 24.8 25 24.6 24.3 24.3]]
New_Zeta_epsi: 4.921185465236567
FDDN Solver Output: [[73.6 35.1 111 72.6 38.4 118 11.6 25.1 25 25.1 24.6 24.2 11.3 24.7 24.9 24.5 24.2 24.2]]
Row ID: 28
HoV: ['S2']
Flow Rate Difference (LTR - FDDN): 0.02437499107302421 cu.m/s
GRADIENT_UPDATE - dw_ih: [array([-0.00207]), array([-0.00144])] <class 'numpy.ndarray'> (2, 1)
GRADIENT_UPDATE - dw_ho [array([0])] <class 'numpy.ndarray'> (1, 1)
f1: 7.785023770675358e-06
Learning Rate 2: 5.536740384450001

CALC NEW WEIGHTS - Learning Rate Used: 5.536740384450001
CALC NEW WEIGHTS - delta_weights_input_i_h_o: [array([-0.0025]), array([-0.0018]), array([0])]
CALC NEW WEIGHTS - Inputs Passed Weights_i_h: [array([0.63]), array([-0.216])] Shape: (2, 1)
CALC NEW WEIGHTS - Inputs Passed Weights_h_o: [array([1])] Shape: (1, 1)
CALC NEW WEIGHTS - new_weights_i_h: [array([0.644]), array([-0.206])]
CALC NEW WEIGHTS - new_weights_h_o: [array([1])]


FDDN Solver Output: [[69 32.9 104 68.1 36 111 10.9 23.5 23.5 23.5 23 22.7 10.6 23.2 23.3 23 22.7 22.7]]
New_Zeta_epsi: 6.2449291164752365
FDDN Solver Output: [[68.9 32.9 104 67.9 35.9 111 10.9 23.5 23.4 23.5 23 22.7 10.6 23.1 23.3 22.9 22.6 22.6]]
Row ID: 15
HoV: ['E2']
Flow Rate Difference (LTR - FDDN): -0.0027834725217820155 cu.m/s

NN output (C_f): 1.1749348569644698
New_Zeta: 4.9852166682863865
FDDN Solver Output: [[73.4 35 111 72.4 38.2 118 11.6 25 25 25 24.5 24.1 11.3 24.6 24.8 24.4 24.1 24.1]]
New_Zeta_epsi: 5.034710421121112
FDDN Solver Output: [[73.2 34.9 110 72.2 38.1 118 11.6 24.9 24.9 25 24.4 24.1 11.3 24.6 24.8 24.4 24.1 24.1]]
Row ID: 28
HoV: ['S2']
Flow Rate Difference (LTR - FDDN): 0.028306942820245262 cu.m/s
GRADIENT_UPDATE - dw_ih: [array([5.14e-05]), array([0.000312])] <class 'numpy.ndarray'> (2, 1)
GRADIENT_UPDATE - dw_ho [array([0])] <class 'numpy.ndarray'> (1, 1)
f1: 1.0751022849408461e-07
Learning Rate 2: 13.12703452803702

CALC NEW WEIGHTS - Learning Rate Used: 

FDDN Solver Output: [[68.9 32.9 104 67.9 35.9 111 10.9 23.5 23.4 23.5 23 22.7 10.6 23.1 23.3 22.9 22.6 22.7]]
Row ID: 15
HoV: ['E2']
Flow Rate Difference (LTR - FDDN): -0.0030650342917037943 cu.m/s
GRADIENT_UPDATE - dw_ih: [array([0.0129]), array([0.0159])] <class 'numpy.ndarray'> (2, 1)
GRADIENT_UPDATE - dw_ho [array([0])] <class 'numpy.ndarray'> (1, 1)
f0: 0.00041920683585228884

CALC NEW WEIGHTS - Learning Rate Used: 1
CALC NEW WEIGHTS - delta_weights_input_i_h_o: [array([0.0129]), array([0.0159]), array([0])]
CALC NEW WEIGHTS - Inputs Passed Weights_i_h: [array([0.645]), array([-0.209])] Shape: (2, 1)
CALC NEW WEIGHTS - Inputs Passed Weights_h_o: [array([1])] Shape: (1, 1)
CALC NEW WEIGHTS - new_weights_i_h: [array([0.632]), array([-0.225])]
CALC NEW WEIGHTS - new_weights_h_o: [array([1])]

NN output (C_f): 1.1455530243758851
New_Zeta: 4.860550351088308
FDDN Solver Output: [[73.8 35.2 111 72.8 38.5 119 11.7 25.2 25.1 25.2 24.6 24.3 11.4 24.8 25 24.6 24.3 24.3]]
New_Zeta_epsi: 4.910

FDDN Solver Output: [[74.1 35.3 112 73 38.6 119 11.7 25.2 25.2 25.2 24.7 24.4 11.4 24.8 25 24.6 24.3 24.3]]
New_Zeta_epsi: 4.85371590224918
FDDN Solver Output: [[73.9 35.2 111 72.8 38.5 119 11.7 25.2 25.1 25.2 24.6 24.3 11.4 24.8 25 24.6 24.3 24.3]]
Row ID: 28
HoV: ['S2']
Flow Rate Difference (LTR - FDDN): 0.02203470223703685 cu.m/s

NN output (C_f): 1.4102726502871898
New_Zeta: 5.983748529858044
FDDN Solver Output: [[69.7 33.2 105 68.7 36.3 112 11 23.7 23.7 23.7 23.2 22.9 10.7 23.4 23.6 23.2 22.9 22.9]]
New_Zeta_epsi: 6.033242282692769
FDDN Solver Output: [[69.5 33.2 105 68.5 36.2 112 11 23.7 23.6 23.7 23.2 22.9 10.7 23.3 23.5 23.1 22.8 22.9]]
Row ID: 15
HoV: ['E2']
Flow Rate Difference (LTR - FDDN): -0.008976436150532563 cu.m/s
GRADIENT_UPDATE - dw_ih: [array([0.0102]), array([0.0127])] <class 'numpy.ndarray'> (2, 1)
GRADIENT_UPDATE - dw_ho [array([0])] <class 'numpy.ndarray'> (1, 1)
f0: 0.00026686412756920944

CALC NEW WEIGHTS - Learning Rate Used: 1
CALC NEW WEIGHTS - delta_weights

FDDN Solver Output: [[69.1 33 104 68.1 36 111 10.9 23.5 23.5 23.6 23 22.7 10.6 23.2 23.4 23 22.7 22.7]]
Row ID: 15
HoV: ['E2']
Flow Rate Difference (LTR - FDDN): -0.004919683464067348 cu.m/s
GRADIENT_UPDATE - dw_ih: [array([-0.00241]), array([-0.000209])] <class 'numpy.ndarray'> (2, 1)
GRADIENT_UPDATE - dw_ho [array([0])] <class 'numpy.ndarray'> (1, 1)
f0: 5.8403919294301156e-06

CALC NEW WEIGHTS - Learning Rate Used: 1
CALC NEW WEIGHTS - delta_weights_input_i_h_o: [array([-0.00241]), array([-0.000209]), array([0])]
CALC NEW WEIGHTS - Inputs Passed Weights_i_h: [array([0.649]), array([-0.336])] Shape: (2, 1)
CALC NEW WEIGHTS - Inputs Passed Weights_h_o: [array([1])] Shape: (1, 1)
CALC NEW WEIGHTS - new_weights_i_h: [array([0.652]), array([-0.336])]
CALC NEW WEIGHTS - new_weights_h_o: [array([1])]

NN output (C_f): 1.448469369837071
New_Zeta: 6.145816172881156
FDDN Solver Output: [[69.2 33 104 68.2 36 111 10.9 23.6 23.5 23.6 23.1 22.8 10.7 23.2 23.4 23 22.7 22.7]]
New_Zeta_epsi: 6.19530

In [21]:
print("MSE_flowloss_history:",MSE_flowloss_hist)

MSE_flowloss_history: [0.0007099331839445402, 0.00034374360436651596, 5.653466168382459e-05, 6.943070779524925e-06, 6.410954463984942e-05, 9.14863904266379e-06, 5.314835743770117e-07, 1.1101648341714236e-05, 2.876693008392939e-05, 7.519230322519686e-05, 0.00035108693975094364, 3.691773571632952e-05, 0.0005247010161618911, 1.0582822339876856e-05, 3.0325104537640353e-05, 6.784074334548151e-05, 5.92249128282294e-05, 5.838572248986195e-07, 8.605924442870328e-05, 0.00010219154218975908, 9.246291057338253e-05, 1.4871067323635211e-06, 0.00010174699237115785, 0.00022993466591236813, 0.0001598292538923325, 9.595626577236974e-05, 8.121616690325343e-05, 5.1477872156893196e-05, 0.00015700938124551292, 0.00022508008454870854, 9.861580291622057e-05, 0.00010246575918282173, 1.749385291393348e-05, 3.1561122438371174e-05, 1.5790891603480157e-06, 0.00046463119100317765, 0.0006006679234128087, 3.577099277009775e-05, 4.7200171037524965e-05, 6.0523929124692734e-05, 0.0004237129758685548, 3.5273842019405916

In [22]:
print("Error_loss_history:",err_loss_hist)

Error_loss_history: [0.001216232750590215, 0.0005888895446661387, 9.685320906027218e-05, 1.189462651232235e-05, 0.00010983023414003619, 1.5673129075602193e-05, 9.105191082440924e-07, 1.9018956437149996e-05, 4.928250051310046e-05, 0.00012881682930592357, 0.0006014698905283862, 6.324617621950982e-05, 0.0008988994662544333, 1.813012184036955e-05, 5.195191059928166e-05, 0.00011622239352542431, 0.00010146205341793154, 1.0002438182209143e-06, 0.00014743369366632404, 0.00017507098309439792, 0.00015840423098609119, 2.547659346580149e-06, 0.00017430939586213524, 0.00039391604379559206, 0.000273813898944945, 0.00016438892517779805, 0.0001391367022968381, 8.819009374944313e-05, 0.0002689829915537736, 0.0003855993444520255, 0.0001689451513817773, 0.00017554076207539735, 2.996985819104363e-05, 5.406941332373809e-05, 2.7052404315661515e-06, 0.0007959899385234931, 0.0010290433201397492, 6.128161623096822e-05, 8.086168550445187e-05, 0.00010368748279500448, 0.0007258902806008791, 6.042991491787795e-06,

In [23]:
cf_df = pd.DataFrame(data=c_f_hist)
cf_df.to_csv('data_output/NN_1CF_Output_FE_LS_miniBatch.csv',index=False)
# cf_df.sort_values(by=['HoV']).reset_index(drop=True)
cf_df.sort_values(by=['ConvergedEpoch']).reset_index(drop=True)

,HoV,c_f_hat,FlowRate_Diff_(LTR-FDDN),ConvergedEpoch,f1,f2
0,A1,1.154844,0.000650,2,5.688675e-05,4.349374e-06
1,C1,0.989247,0.000168,3,1.314433e-05,6.752414e-07
2,S3,1.015988,0.001021,4,4.830228e-06,-6.124876e-07
3,S4,1.087005,-0.000490,5,4.485633e-06,-3.038227e-08
4,S1,1.312477,0.000221,6,1.320454e-05,2.912037e-07
5,M2,1.215954,-0.001007,6,1.320454e-05,2.912037e-07
6,C5,1.259022,-0.001534,7,6.448680e-05,7.752511e-06
7,E1,1.013127,-0.000855,8,5.985424e-07,-3.253283e-08
8,A5,1.007067,0.000717,11,1.192970e-07,2.039706e-08
9,C2,1.267696,0.000604,14,9.091914e-06,5.479751e-07


### Plot Training Loss

In [24]:
hover1    = [('Epoch', '@x'),('MSE FlowLoss', '@y') ]
p1 = figure(width=1600, tooltips=hover1)
p1.line(list(range(1, len(MSE_flowloss_hist)+1)), MSE_flowloss_hist, line_width=2, color='blue', alpha=0.8)
p1.yaxis.axis_label = "MSE FlowRate Loss"
p1.xaxis.axis_label = "Epochs"
p1.legend.click_policy="hide"
show(p1)

In [25]:
hover2    = [('Epoch', '@x'),('Error', '@y') ]
p2 = figure(width=1600, tooltips=hover2)
p2.line(list(range(1, len(err_loss_hist)+1)), err_loss_hist, line_width=2, color='red', alpha=0.8)
p2.yaxis.axis_label = "Error (Loss Function)"
p2.xaxis.axis_label = "Epochs"
show(p2)